In [1]:
#installing transformers in colab only -- remove if using on diff tpu
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 15.5MB/s 
     |████████████████████████████████| 3.0MB 16.2MB/s 
     |████████████████████████████████| 890kB 44.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e9933637657d6f3fd55b01f145b97c611a32ff6b524a64d78ffdd4eb6d7e7769
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
#reading suicide data csv, batch_1 csv is all relevant data for training
df = pd.read_csv('https://raw.githubusercontent.com/ayaanzhaque/SuiSense/master/data/data_for_model.csv')
og_batch_1 = df[['selftext', 'is_suicide']]
batch_1_start = og_batch_1.head(125)
batch_1_end = og_batch_1.tail(125)
batch_1 = pd.concat([batch_1_start, batch_1_end], ignore_index=True)
print(batch_1)



                                              selftext  is_suicide
0    We understand that most people who reply immed...           0
1    Welcome to /r/depression's check-in post - a p...           0
2    I've been feeling really depressed and lonely ...           0
3    I literally broke down crying and asked to go ...           0
4    Any kind soul want to give a depressed person ...           0
..                                                 ...         ...
245  It's so hard just to get out of bed every morn...           1
246  I have a low IQ. Twice I've been tested and bo...           1
247                  I don't feel useless, im useless.           1
248  I wish there was some way I could will myself ...           1
249  i want to die but most of all i think i want t...           1

[250 rows x 2 columns]


In [4]:
#how many ppl are suicidal(1) vs depressed(0)
batch_1['is_suicide'].value_counts()

1    125
0    125
Name: is_suicide, dtype: int64

In [5]:
#initializing bert base model in the program
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Loading pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [6]:
#tokenizing the actual sentences
tokenized = batch_1['selftext'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512)))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
np.array(padded).shape

(250, 512)

In [9]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(250, 512)

In [10]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [11]:
features = batch_1["selftext"]
print(features)

0      We understand that most people who reply immed...
1      Welcome to /r/depression's check-in post - a p...
2      I've been feeling really depressed and lonely ...
3      I literally broke down crying and asked to go ...
4      Any kind soul want to give a depressed person ...
                             ...                        
245    It's so hard just to get out of bed every morn...
246    I have a low IQ. Twice I've been tested and bo...
247                    I don't feel useless, im useless.
248    I wish there was some way I could will myself ...
249    i want to die but most of all i think i want t...
Name: selftext, Length: 250, dtype: object


In [12]:
labels = batch_1['is_suicide']
print(labels)

0      0
1      0
2      0
3      0
4      0
      ..
245    1
246    1
247    1
248    1
249    1
Name: is_suicide, Length: 250, dtype: int64


In [13]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [14]:
#currently logistic regression. When we find a neural network, substitute it here and onwards
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

ValueError: ignored

In [ ]:
lr_clf.score(test_features, test_labels)

In [ ]:
#comparing above score with dummy classifier score
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))